## Word2Vec Algorithm for Word Embeddings (Skip gram)

In [ ]:
import tensorflow as tf
import nltk
nltk.download('punkt') #for dealing with punctuation. it can handle cases like "!!!!" or "..."
from nltk.tokenize import sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
import numpy as np

2025-07-29 17:31:06.094125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753810266.117125   48741 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753810266.124104   48741 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Familial Relationships Corpus

In [2]:
corpus = [
    # Family & Royalty
    "The king is a man.",
    "The queen is a woman.",
    "The prince is the son of the king and queen.",
    "The princess is the daughter of the king and queen.",
    "The man is strong and brave.",
    "The woman is wise and kind.",
    "The king and queen rule the kingdom.",
    "The prince loves the princess.",
    "The princess adores the prince.",
    "The king has a crown and a throne.",
    "The queen wears a beautiful crown.",
    "The prince has a shiny sword.",
    "The princess wears a delicate necklace.",
    "The man hunts in the forest.",
    "The woman visits the village.",
    "He is a loyal man.",
    "She is an intelligent woman.",
    "The king commands his army.",
    "The queen advises the king wisely.",
    "The prince trains with knights.",
    "The princess studies magic.",
    "The king holds a scepter.",
    "The queen wears a tiara.",
    "The prince dreams of adventures.",
    "The princess dreams of peace.",
    "The king hosts a feast.",
    "The queen bakes bread.",
    "The prince learns to read maps.",
    "The princess tends the flowers.",
    "The prince helps his father.",
    "The princess helps her mother.",
    "The king signs the law.",
    "The queen attends meetings.",
    "The prince and princess play in the garden.",
    "The prince rides a horse.",
    "The princess rides a white horse.",
    "He respects the king.",
    "She follows the queen’s advice.",
    
    # Animals & Descriptions
    "The hare runs very fast.",
    "The rabbit is similar to the hare.",
    "The rabbit lives in a burrow underground.",
    "The hare lives in tall grass.",
    "The dog barks loudly.",
    "The cat sleeps on the roof.",
    "The lion is the king of the jungle.",
    "The tiger is fierce and strong.",
    "The dog chases the cat.",
    "The cat climbs the tree.",
    "The lion roars in the jungle.",
    "The tiger stalks its prey.",
    "The wolf hunts at night.",
    "The rabbit hides in the bushes.",
    "The hare hides from the wolf.",
    "The dog follows the man.",
    "The cat follows the woman.",
    "The lion leads the pride.",
    "The tiger prowls silently.",
    "The wolf watches the forest.",
    "The rabbit hops quickly.",
    "The hare jumps far.",
    "The dog wags its tail.",
    "The cat purrs softly.",
    "The lion sleeps in the sun.",
    "The tiger drinks from the river.",
    "The rabbit hides in the tall grass.",
    "The hare listens carefully.",
    "The dog guards the castle.",
    "The cat watches quietly.",
    "The rabbit and hare eat grass.",
    "The wolf chases the deer.",
    "The fox is cunning and quick.",
    "The bear is strong and large.",
    "The eagle flies high in the sky.",
    "The owl hunts at night.",
    
    # Actions & Verbs
    "The man builds a house.",
    "The woman plants flowers.",
    "The man carries wood.",
    "The woman cooks meals.",
    "He rides a horse.",
    "She rides a carriage.",
    "The man repairs the fence.",
    "The woman sews clothes.",
    "He watches the stars.",
    "She reads a book.",
    "The prince trains his horse.",
    "The princess waters the garden.",
    "The man hunts deer.",
    "The woman gathers berries.",
    "The dog chases squirrels.",
    "The cat climbs trees.",
    "The man works in the fields.",
    "The woman teaches children.",
    "The prince learns sword fighting.",
    "The princess learns to read and write.",
    "The prince brings gifts.",
    "The princess makes a delicious cake.",
    "The wolf watches the forest.",
    "The king commands his army.",
    "The queen advises the king wisely.",
    "The prince helps his father.",
    "The princess helps her mother.",
    
    # Descriptive Adjectives & Qualities
    "The man is brave.",
    "The woman is kind.",
    "The king is powerful.",
    "The queen is wise.",
    "The prince is young.",
    "The princess is gentle.",
    "The hare is fast.",
    "The rabbit is soft.",
    "The dog is loyal.",
    "The cat is curious.",
    "The lion is fierce.",
    "The tiger is strong.",
    "The wolf is fearless.",
    "The fox is clever.",
    "The bear is large.",
    "The eagle is sharp-eyed.",
    "The owl is silent.",
    "The knight is brave.",
    "The advisor is clever.",
    "The king is just.",
    "The queen is graceful.",
    "The prince is courageous.",
    "The princess is beautiful.",
    "The dog is playful.",
    "The cat is sleepy.",
    "The man is hardworking.",
    "The woman is creative.",
    
    # More Family Relations & Roles
    "The father works hard.",
    "The mother cares for the children.",
    "The brother plays football.",
    "The sister reads books.",
    "The son studies math.",
    "The daughter paints pictures.",
    "The grandfather tells stories.",
    "The grandmother bakes cookies.",
    "The uncle visits often.",
    "The aunt is kind.",
    "The cousin is funny.",
    "The nephew helps around.",
    "The niece loves animals.",
    "The family celebrates festivals.",
    "The parents love their children.",
    
    # More Animals & Nature
    "The horse gallops across the field.",
    "The cow grazes in the meadow.",
    "The sheep follows the herd.",
    "The goat climbs the hill.",
    "The duck swims in the pond.",
    "The fish swims in the river.",
    "The frog jumps from lily pad to lily pad.",
    "The butterfly flies among flowers.",
    "The bee collects nectar.",
    "The ant carries food.",
    "The spider spins a web.",
    "The eagle soars over the mountains.",
    "The rabbit nibbles on carrots.",
    "The fox hides in the forest.",
    "The bear fishes in the stream.",

    # Occupations & Activities
    "The doctor helps the sick.",
    "The teacher educates the students.",
    "The farmer grows crops.",
    "The baker makes bread.",
    "The singer sings beautifully.",
    "The dancer performs on stage.",
    "The soldier protects the country.",
    "The artist paints landscapes.",
    "The writer writes stories.",
    "The chef cooks delicious meals.",
    "The carpenter builds furniture.",
    "The police officer keeps peace.",
    "The firefighter fights fires.",
    "The engineer designs bridges.",
    "The pilot flies airplanes.",
    
    # Mixed Semantic Relationships
    "The prince and princess attend the ball.",
    "The king and queen rule justly.",
    "The hare and rabbit avoid the wolf.",
    "He is a brave knight.",
    "She is a clever advisor.",
    "The dog and cat live together.",
    "The fox and wolf hunt at night.",
    "The farmer and baker work in the village.",
    "The teacher and doctor help people.",
    "The singer and dancer perform at the festival.",
    "The lion and tiger are big cats.",
    "The eagle and owl are birds of prey.",

    "A man is the husband of a woman.",
    "A woman is the wife of a man.",
    "A father is the husband of a mother.",
    "A mother is the wife of a father.",
    "A grandfather is the husband of a grandmother.",
    "A grandmother is the wife of a grandfather.",
    "An uncle is the brother of a father.",
    "An aunt is the sister of a mother.",
    "A boy is the son of a man and a woman.",
    "A girl is the daughter of a man and a woman.",
    "A father is a male parent.",
    "A mother is a female parent.",
    "A grandfather is a male grandparent.",
    "A grandmother is a female grandparent.",
    "A son is a male child.",
    "A daughter is a female child.",
    "A brother is a male sibling.",
    "A sister is a female sibling.",
    "A nephew is the son of a sibling.",
    "A niece is the daughter of a sibling.",
    "A man is an adult male.",
    "A woman is an adult female.",
    "A king is a male ruler.",
    "A queen is a female ruler.",
    "A prince is the son of a king.",
    "A princess is the daughter of a king.",
    "A husband loves his wife.",
    "A wife loves her husband.",
    "A grandfather loves his grandchildren.",
    "A grandmother loves her grandchildren.",
    "A father teaches his son.",
    "A mother teaches her daughter.",
    "A father guides his daughter.",
    "A mother guides her son.",
    "A father plays with his children.",
    "A mother plays with her children.",
    "A father hugs his daughter.",
    "A mother hugs her son.",
    "A grandfather tells stories to his grandson.",
    "A grandmother tells stories to her granddaughter.",
    "A man respects his wife.",
    "A woman respects her husband.",
    "A son learns from his father.",
    "A daughter learns from her mother.",
    "A boy grows into a man.",
    "A girl grows into a woman.",
    "A brother protects his sister.",
    "A sister helps her brother.",
    "A husband supports his wife.",
    "A wife supports her husband.",
    "A grandfather shares wisdom with his grandson.",
    "A grandmother shares wisdom with her granddaughter.",
    "A man cares for his family.",
    "A woman cares for her family.",
    "A king rules beside his queen.",
    "A queen rules beside her king.",
    "A son looks up to his father.",
    "A daughter looks up to her mother.",
    "A grandfather gives advice to his grandson.",
    "A grandmother gives advice to her granddaughter.",
    "A father works for his family.",
    "A mother nurtures her family.",
    "A boy is raised by his father and mother.",
    "A girl is raised by her mother and father.",
    "A man leads his household.",
    "A woman organizes her home.",
    "A father builds a house.",
    "A mother decorates the home.",
    "A grandfather repairs furniture.",
    "A grandmother sews clothes.",
    "A nephew visits his uncle.",
    "A niece visits her aunt.",
    "A cousin plays with his cousin.",
    "A niece helps her aunt cook.",
    "A nephew helps his uncle clean.",
    "A husband kisses his wife.",
    "A wife kisses her husband.",
    "A man marries a woman.",
    "A woman marries a man.",
    "A grandfather sits beside his grandmother.",
    "A grandmother sits beside her grandfather."
    
    "The mother teaches her son to read.",
    "The father teaches his daughter to ride.",
    "The mother helps her daughter with homework.",
    "The father helps his son fix the bike.",
    "The son listens to his mother’s advice.",
    "The daughter listens to her father’s stories.",
    "The mother scolds her son gently.",
    "The father praises his daughter proudly.",
    "The son admires his father’s strength.",
    "The daughter admires her mother’s kindness.",
    "The mother reads a book to her child.",
    "The father tells a story to his children.",
    "The mother cooks for her children every day.",
    "The father drives his daughter to school.",
    "The son hugs his mother tightly.",
    "The daughter kisses her father on the cheek.",
    "The mother wakes her son in the morning.",
    "The father drops his daughter at school.",
    "The child runs to their mother.",
    "The child holds their father's hand.",
    "The grandmother bakes cookies for her grandchildren.",
    "The grandfather tells stories to his grandson.",
    "The granddaughter helps her grandmother in the kitchen.",
    "The grandson helps his grandfather in the garden.",
    "The grandmother knits a sweater for her granddaughter.",
    "The grandfather builds a toy for his grandson.",
    "The granddaughter kisses her grandfather.",
    "The grandson hugs his grandmother.",
    "The grandfather teaches his grandson to fish.",
    "The grandmother teaches her granddaughter to sew.",
    "The grandchildren visit their grandparents on Sundays.",
    "The grandmother comforts her crying grandson.",
    "The grandfather laughs with his granddaughter.",
    "The grandson listens to his grandmother’s stories.",
    "The granddaughter learns baking from her grandmother.",
    "The grandmother teaches family recipes to her granddaughter.",
    "The grandfather shows his photo albums to his grandson.",
    "The grandson plays cards with his grandfather.",
    "The granddaughter sings a song for her grandmother.",
    "The grandchildren help their grandparents clean the house.",
    "The brother protects his younger sister.",
    "The sister helps her older brother with homework.",
    "The brother teases his sister playfully.",
    "The sister laughs with her brother.",
    "The brother shares his toys with his sister.",
    "The sister gives her brother a handmade gift.",
    "The siblings play together in the yard.",
    "The younger brother follows his older sister everywhere.",
    "The elder sister teaches her brother to draw.",
    "The younger sister sings with her elder brother.",
    "The brother lends his book to his sister.",
    "The sister waits for her brother after school.",
    "The siblings help their parents clean the house.",
    "The brother builds a fort for his sister.",
    "The sister decorates the fort with her brother.",
    "The brother carries his sister’s schoolbag.",
    "The sister tells her brother a secret.",
    "The brother confides in his sister.",
    "The siblings argue but forgive each other.",
    "The sister makes lunch for her brother.",
    "The uncle teaches his nephew to play guitar.",
    "The aunt bakes cookies for her niece.",
    "The nephew visits his aunt every weekend.",
    "The niece brings flowers for her uncle.",
    "The uncle takes his niece to the park.",
    "The aunt takes her nephew to the museum.",
    "The nephew helps his aunt carry groceries.",
    "The niece sings a song for her uncle.",
    "The uncle builds a kite for his nephew.",
    "The aunt reads a story to her niece.",
    "The niece hugs her uncle tightly.",
    "The nephew laughs with his aunt.",
    "The uncle tells jokes to his niece.",
    "The aunt gives a gift to her nephew.",
    "The nephew waters plants with his aunt.",
    "The niece paints with her uncle.",
    "The uncle visits his niece’s school.",
    "The aunt claps for her nephew’s performance.",
    "The niece thanks her aunt for the dress.",
    "The nephew shows his drawing to his uncle.",
    "The cousin helps her younger cousin with math.",
    "The older cousin plays football with his cousin.",
    "The cousin shares his toys with his cousin.",
    "The cousins visit their grandparents together.",
    "The younger cousin follows her older cousin around.",
    "The older cousin reads to his cousin at bedtime.",
    "The cousin teaches his cousin to ride a bicycle.",
    "The cousin laughs with her cousin at jokes.",
    "The cousin invites her cousin to her birthday.",
    "The cousins go hiking with their family.",
    "The cousin lends her dress to her cousin.",
    "The cousin explains a game to his cousin.",
    "The cousins play in the backyard all day.",
    "The cousin helps his cousin fly a kite.",
    "The cousin joins her cousin in a dance.",
    "The cousin tells a ghost story to his cousin.",
    "The cousin and her cousin decorate cookies.",
    "The cousins collect stones together.",
    "The cousin claps for his cousin on stage.",
    "The cousin comforts her cousin when sad.",
    "The family eats dinner together every night.",
    "The parents love their children dearly.",
    "The siblings respect each other.",
    "The grandparents bless the family.",
    "The parents teach their children discipline.",
    "The family celebrates holidays together.",
    "The mother supports her husband and children.",
    "The father protects his wife and children.",
    "The family sits together in the garden.",
    "The father reads the newspaper aloud to his family.",
    "The mother sings lullabies to her children.",
    "The daughter dances for the entire family.",
    "The family tells jokes after dinner.",
    "The grandparents listen to the children sing.",
    "The mother thanks her children for helping.",
    "The father congratulates his son on good grades.",
    "The sister shares her artwork with the family.",
    "The family claps for the daughter’s performance.",
    "The brother plays music for the family.",
    "The whole family takes a walk in the evening.",

    "A king is the husband of a queen.",
"A queen is the wife of a king.",
"A king rules the kingdom with his queen.",
"A queen rules the kingdom with her king.",
"A prince is the son of a king.",
"A princess is the daughter of a king.",
"A king commands his soldiers.",
"A queen leads her advisors.",
"A prince learns from the king.",
"A princess learns from the queen.",
"A king and queen sit on the throne together.",
"A queen and king host a royal feast.",
"A prince respects the king.",
"A princess respects the queen.",
"A king loves his queen.",
"A queen loves her king.",
"A prince rides beside his father.",
"A princess rides beside her mother.",
"A prince receives gifts from the king.",
"A princess receives gifts from the queen.",
"A king wears a crown.",
"A queen wears a tiara.",
"A prince holds a sword.",
"A princess holds a wand.",
"A king addresses the court.",
"A queen addresses the court.",
"A prince prepares for battle.",
"A princess prepares for a ceremony.",
"A king consults the queen.",
"A queen consults the king.",
"A prince dances with a princess.",
"A princess dances with a prince.",
"A king honors the queen.",
"A queen honors the king.",

    "A father loves his daughter.",
"A mother loves her son.",
"A father teaches his son math.",
"A mother teaches her daughter science.",
"A father and son build a treehouse.",
"A mother and daughter bake a cake.",
"A father praises his son.",
"A mother praises her daughter.",
"A son plays with his father.",
"A daughter plays with her mother.",
"A father carries his daughter.",
"A mother carries her son.",
"A father teaches his son to ride a bicycle.",
"A mother teaches her daughter to swim.",
"A father reads to his daughter.",
"A mother reads to her son.",
"A father drives his son to school.",
"A mother drives her daughter to school.",
"A son asks his father questions.",
"A daughter asks her mother questions.",
"A father and mother protect their children.",
"A son helps his father in the yard.",
"A daughter helps her mother in the kitchen.",
"A father and daughter walk together.",
"A mother and son walk together.",
"A father laughs with his son.",
"A mother laughs with her daughter.",
"A son learns from his mother.",
"A daughter learns from her father.",
"A father supports his daughter’s dreams.",
"A mother supports her son’s goals.",

    "A grandfather teaches his grandson to fish.",
"A grandmother teaches her granddaughter to sew.",
"A grandson visits his grandfather.",
"A granddaughter visits her grandmother.",
"A grandfather tells stories to his grandchildren.",
"A grandmother reads stories to her grandchildren.",
"A grandfather hugs his granddaughter.",
"A grandmother hugs her grandson.",
"A grandfather bakes with his granddaughter.",
"A grandmother cooks with her grandson.",
"A grandson listens to his grandfather.",
"A granddaughter listens to her grandmother.",
"A grandfather helps his grandson build a model.",
"A grandmother helps her granddaughter paint a picture.",
"A grandfather teaches history to his grandson.",
"A grandmother teaches gardening to her granddaughter.",
"A grandson shares candy with his grandfather.",
"A granddaughter shares drawings with her grandmother.",
"A grandfather and grandmother sit on the porch.",
"A grandson and granddaughter play in the garden.",
"A grandfather fixes the swing for his grandchildren.",
"A grandmother makes tea for her grandchildren.",
"A grandson plays cards with his grandfather.",
"A granddaughter knits with her grandmother.",
"A grandfather smiles at his granddaughter.",
"A grandmother smiles at her grandson.",
"A grandfather builds a birdhouse with his grandson.",
"A grandmother bakes cookies with her granddaughter.",
"A grandfather tells jokes to his granddaughter.",
"A grandmother tells stories to her grandson.",

    "A brother helps his sister with homework.",
"A sister helps her brother with homework.",
"A brother plays football with his sister.",
"A sister plays chess with her brother.",
"A brother and sister walk to school together.",
"A sister and brother ride bicycles together.",
"A brother shares his snack with his sister.",
"A sister shares her crayons with her brother.",
"A brother teases his sister.",
"A sister teases her brother.",
"A brother hugs his sister after a fight.",
"A sister hugs her brother after a fight.",
"A brother learns from his sister.",
"A sister learns from her brother.",
"A brother protects his younger sister.",
"A sister protects her younger brother.",
"A brother talks to his sister about school.",
"A sister talks to her brother about books.",
"A brother helps his sister clean her room.",
"A sister helps her brother clean his room.",
"A brother and sister visit their grandparents together.",
"A sister and brother plant a tree together.",
"A brother laughs with his sister.",
"A sister laughs with her brother.",
"A brother writes a letter to his sister.",
"A sister writes a poem for her brother.",
"A brother sings with his sister.",
"A sister dances with her brother.",
"A brother supports his sister’s dreams.",
"A sister supports her brother’s goals.",

    "A husband supports his wife.",
"A wife supports her husband.",
"A husband loves his wife deeply.",
"A wife loves her husband dearly.",
"A husband and wife share a home.",
"A wife and husband raise children together.",
"A husband works for his family.",
"A wife cares for her family.",
"A husband and wife go on a walk together.",
"A wife and husband attend a party together.",
"A husband cooks dinner for his wife.",
"A wife bakes a cake for her husband.",
"A husband writes a note to his wife.",
"A wife writes a poem for her husband.",
"A husband praises his wife’s work.",
"A wife praises her husband’s effort.",
"A husband holds his wife’s hand.",
"A wife holds her husband’s hand.",
"A husband tells jokes to his wife.",
"A wife laughs at her husband’s jokes.",
"A husband listens to his wife.",
"A wife listens to her husband.",
"A husband comforts his wife.",
"A wife comforts her husband.",
"A husband gives flowers to his wife.",
"A wife gives a book to her husband.",
"A husband helps his wife carry groceries.",
"A wife helps her husband fix the car.",
"A husband thanks his wife.",
"A wife thanks her husband.",

    "An uncle visits his nephew on weekends.",
"An aunt visits her niece on weekends.",
"An uncle teaches his nephew to paint.",
"An aunt teaches her niece to sing.",
"An uncle plays catch with his nephew.",
"An aunt braids her niece’s hair.",
"A nephew hugs his uncle goodbye.",
"A niece hugs her aunt goodbye.",
"An uncle gives advice to his nephew.",
"An aunt gives advice to her niece.",
"A nephew listens to his uncle.",
"A niece listens to her aunt.",
"A nephew thanks his uncle for a gift.",
"A niece thanks her aunt for a dress.",
"A nephew shows his uncle a drawing.",
"A niece shows her aunt a painting.",
"An uncle takes his nephew to the zoo.",
"An aunt takes her niece to the museum.",
"An uncle buys a toy for his nephew.",
"An aunt buys a book for her niece.",
"A nephew plays games with his uncle.",
"A niece plays music with her aunt.",
"An uncle helps his nephew build a model.",
"An aunt helps her niece bake cookies.",
"A nephew laughs with his uncle.",
"A niece laughs with her aunt.",
"An uncle tells a story to his nephew.",
"An aunt tells a story to her niece.",
"An uncle teaches his nephew guitar.",
"An aunt teaches her niece piano.",

    "A grandfather walks with his grandson in the park.",
"A grandmother walks with her granddaughter in the garden.",
"A grandfather teaches chess to his grandson.",
"A grandmother teaches knitting to her granddaughter.",
"A grandson brings flowers to his grandmother.",
"A granddaughter brings books to her grandfather.",
"A grandfather watches his grandson play football.",
"A grandmother watches her granddaughter dance.",
"A grandson eats lunch with his grandmother.",
"A granddaughter eats dinner with her grandfather.",
"A grandfather tells jokes to his granddaughter.",
"A grandmother tells jokes to her grandson.",
"A grandfather listens to his grandson's story.",
"A grandmother listens to her granddaughter's song.",
"A grandson helps his grandfather paint the fence.",
"A granddaughter helps her grandmother wash dishes.",
"A grandfather repairs the birdhouse with his grandson.",
"A grandmother plants flowers with her granddaughter.",
"A grandfather and grandmother visit their grandchildren together.",
"A grandson gives a gift to his grandmother.",
"A granddaughter gives a drawing to her grandfather.",
"A grandson thanks his grandmother for the cookies.",
"A granddaughter thanks her grandfather for the book.",
"A grandfather explains history to his grandson.",
"A grandmother explains traditions to her granddaughter.",


    "An uncle teaches his nephew how to fish.",
"An aunt teaches her niece how to cook.",
"An uncle laughs with his nephew in the yard.",
"An aunt laughs with her niece in the kitchen.",
"A nephew tells a story to his uncle.",
"A niece tells a story to her aunt.",
"An uncle watches a movie with his nephew.",
"An aunt watches a movie with her niece.",
"An uncle helps his nephew with homework.",
"An aunt helps her niece with homework.",
"A nephew gives a card to his uncle.",
"A niece gives a drawing to her aunt.",
"An uncle and aunt visit their niece and nephew.",
"A niece sits beside her aunt at dinner.",
"A nephew sits beside his uncle at lunch.",
"An uncle listens to his nephew’s ideas.",
"An aunt listens to her niece’s plans.",
"A nephew learns to paint from his uncle.",
"A niece learns to bake from her aunt.",
"A niece hugs her aunt tightly.",
"A nephew hugs his uncle happily.",
"An uncle reads a comic with his nephew.",
"An aunt reads a storybook with her niece.",
"An uncle shows his nephew old photos.",
"An aunt shows her niece family recipes.",

    "A son helps his mother in the garden.",
"A daughter helps her father in the workshop.",
"A son walks to school with his father.",
"A daughter walks to school with her mother.",
"A son writes a letter to his father.",
"A daughter writes a poem to her mother.",
"A son shares a joke with his mother.",
"A daughter shares a joke with her father.",
"A son listens to his mother’s advice.",
"A daughter listens to her father’s advice.",
"A son draws a picture for his father.",
"A daughter draws a card for her mother.",
"A son talks with his mother every evening.",
"A daughter talks with her father every morning.",
"A son asks his mother questions about animals.",
"A daughter asks her father questions about space.",
"A son enjoys cooking with his mother.",
"A daughter enjoys building with her father.",
"A son gives a gift to his mother.",
"A daughter gives a gift to her father.",
"A son carries groceries for his mother.",
"A daughter carries books for her father.",
"A son plays board games with his mother.",
"A daughter plays chess with her father.",

    "A husband and wife walk together in the evening.",
"A wife and husband drink tea on the balcony.",
"A husband gives a necklace to his wife.",
"A wife gives a watch to her husband.",
"A husband helps his wife with the laundry.",
"A wife helps her husband fix the chair.",
"A husband and wife talk about their children.",
"A wife and husband plan a holiday together.",
"A husband reads a book with his wife.",
"A wife reads a newspaper with her husband.",
"A husband listens to his wife’s story.",
"A wife listens to her husband’s ideas.",
"A husband supports his wife in difficult times.",
"A wife supports her husband at work.",
"A husband smiles at his wife.",
"A wife smiles at her husband.",
"A husband and wife visit friends together.",
"A wife and husband prepare dinner together.",
"A husband shares coffee with his wife.",
"A wife shares breakfast with her husband.",
"A husband thanks his wife for her help.",
"A wife thanks her husband for his support.",
"A husband compliments his wife’s cooking.",
"A wife compliments her husband’s effort.",


    "A king leads his kingdom with strength.",
"A queen leads her kingdom with wisdom.",
"A prince trains under the king.",
"A princess trains under the queen.",
"A king wears a golden robe.",
"A queen wears a silver gown.",
"A prince studies maps with the king.",
"A princess studies books with the queen.",
"A king addresses his court.",
"A queen addresses her court.",
"A prince and princess ride horses together.",
"A king and queen bless their children.",
"A prince is loyal to the king.",
"A princess is loyal to the queen.",
"A king invites nobles to the palace.",
"A queen invites guests to the garden.",
"A prince hunts with the king.",
"A princess gathers herbs with the queen.",
"A prince tells stories to the knights.",
"A princess tells stories to the children.",
"A king rules the castle.",
"A queen decorates the castle.",
"A prince guards the castle gate.",
"A princess waters the royal plants.",

    "The father works hard to provide for his family.",
    "The mother lovingly cares for her children.",
    "The brother helps his sister with homework.",
    "The sister supports her brother during difficult times.",
    "The son respects his parents greatly.",
    "The daughter spends time with her grandparents.",
    "The grandfather tells stories of old adventures.",
    "The grandmother bakes cookies every Sunday.",
    "The uncle visits the family during holidays.",
    "The aunt watches over her nieces attentively.",
    "The cousin plays games with the children.",
    "The nephew helps his uncle in the garden.",
    "The niece enjoys reading books with her aunt.",
    "Parents nurture their children with patience and love.",
    "Siblings share secrets and laugh together.",
    "The family gathers around the fireplace in winter.",
    "The father teaches his son how to fish.",
    "The mother teaches her daughter to cook.",
    "The brother and sister walk the dog together.",
    "The father reads bedtime stories to his children.",
    "The mother sews clothes for her family.",
    "Grandparents spoil their grandchildren with treats.",
    "The uncle tells jokes that make everyone laugh.",
    "The aunt helps prepare meals for the family gathering.",
    "The cousins explore the woods behind their house.",
    "The nephew practices sports with his father.",
    "The niece paints pictures with her grandmother.",
    "Parents celebrate their children’s achievements gladly.",
    "Siblings share chores to help their parents.",
    "The family works together to clean the house.",
    "The father repairs broken toys for his kids.",
    "The mother organizes birthday parties for the children.",
    "The brother teaches his sister how to ride a bike.",
    "The sister comforts her brother after a bad dream.",
    "The son helps his mother in the kitchen.",
    "The daughter assists her father in the workshop.",
    "The grandfather teaches his grandchildren gardening.",
    "The grandmother plants flowers in the family garden.",
    "The uncle mentors his nephew in carpentry.",
    "The aunt reads stories to her nieces every night.",
    "The cousins build a treehouse together.",
    "The nephew surprises his aunt with handmade gifts.",
    "The niece sings songs to entertain her family.",
    "Parents encourage their children to study hard.",
    "Siblings support each other through challenges.",
    "The family cooks dinner together on weekends.",
    "The father teaches respect and kindness to his children.",
    "The mother listens patiently to her children’s stories.",
    "Grandparents share wisdom with the younger generation.",
    "The uncle shows his nephew how to cast a fishing line.",
    "The aunt teaches her nieces to knit scarves.",
    "The cousins organize a family picnic in the park.",
    "The nephew helps his uncle fix the car.",
    "The niece decorates the house for family celebrations.",
    "Parents spend quality time playing with their children.",
    "Siblings resolve conflicts and make up quickly.",
    "The family enjoys movie nights around the fireplace.",
    "The father coaches his son’s soccer team.",
    "The mother helps her daughter practice piano.",
    "The brother surprises his sister with a gift.",
    "The sister bakes cookies with her brother.",
    "The son mows the lawn for the family.",
    "The daughter waters the plants in the garden.",
    "The grandfather teaches traditional dances to his grandchildren.",
    "The grandmother knits sweaters for her family.",
    "The uncle shares family history stories with the children.",
    "The aunt bakes pies for the family gathering.",
    "The cousins explore nearby trails on their bikes.",
    "The nephew builds birdhouses with his uncle.",
    "The niece helps her grandmother with gardening chores.",
    "Parents celebrate birthdays with joy and laughter.",
    "Siblings learn valuable lessons from each other.",
    "The family fixes the old fence together.",
    "The father reads poems to his children before bed.",
    "The mother teaches her children good manners.",
    "Grandparents host family reunions every summer.",
    "The uncle entertains the children with magic tricks.",
    "The aunt organizes craft activities for her nieces.",
    "The cousins share snacks during their playtime.",
    "The nephew creates drawings for his family.",
    "The niece writes letters to her grandparents.",
    "Parents set rules to keep their children safe.",
    "Siblings plan surprises for their parents.",
    "The family celebrates holidays with special meals.",
    "The father helps his son with school projects.",
    "The mother encourages her daughter to pursue her dreams.",
    "The brother and sister play musical instruments together.",
    "The son offers to clean the garage.",
    "The daughter arranges flowers for the dining table.",
    "The grandfather tells legends of their ancestors.",
    "The grandmother teaches traditional recipes to the family.",
    "The uncle takes his nephew fishing by the lake.",
    "The aunt reads fairy tales to the children.",
    "The cousins organize sports competitions on weekends.",
    "The nephew surprises his family with a song.",
    "The niece creates handmade cards for family occasions.",
    "Parents guide their children through important life choices.",
    "Siblings share responsibilities around the house.",
    "The family enjoys outdoor adventures together.",
    "The father teaches kindness through his actions.",
    "The mother listens to her children’s hopes and fears.",
    "Grandparents provide comfort and support to the family.",
    "The uncle shares skills like woodworking with his nephew.",
    "The aunt hosts tea parties for her nieces.",
    "The cousins help each other with homework.",
    "The nephew assists his uncle in the kitchen.",
    "The niece bakes treats with her grandmother.",
    "Parents encourage teamwork among siblings.",
    "Siblings bond over shared interests and hobbies.",
    "The family plants trees to improve their neighborhood.",
    "The father mentors his son in sportsmanship.",
    "The mother encourages her daughter’s creativity.",
    "The brother shares books with his sister.",
    "The sister teaches her brother how to swim.",
    "The son cares for the family pet.",
    "The daughter helps with household chores.",
    "The grandfather tells bedtime stories filled with morals.",
    "The grandmother knits blankets to keep her family warm.",
    "The uncle guides his nephew in learning new skills.",
    "The aunt reads books to the children every evening.",
    "The cousins explore the countryside during school breaks.",
    "The nephew builds models of old castles.",
    "The niece decorates the home for festivals.",
    "Parents set examples of honesty and respect.",
    "Siblings invite friends to family gatherings.",
    "The family attends religious ceremonies together.",
    "The father teaches his children about nature.",
    "The mother shares family traditions with her children.",
    "Grandparents pass down heirlooms to their grandchildren.",
    "The uncle teaches the nephew how to cook simple meals.",
    "The aunt organizes family photo albums.",
    "The cousins embark on hikes exploring the local forest.",
    "The nephew surprises his family with art projects.",
    "The niece composes poems about her loved ones.",
    "Parents celebrate every success, big or small.",
    "Siblings comfort each other during tough times.",
    "The family travels together during holidays.",
    "The father helps his son repair a bicycle.",
    "The mother teaches her daughter to sew.",
    "The brother and sister decorate the house for celebrations.",
    "The son learns woodworking from his father.",
    "The daughter plants a vegetable garden.",
    "The grandfather teaches the family traditional songs.",
    "The grandmother tells stories of their cultural heritage.",
    "The uncle spends weekends teaching his nephew archery.",
    "The aunt organizes birthday parties with creative themes.",
    "The cousins share meals and laughter at every gathering.",
    "The nephew brings flowers for his grandmother.",
    "The niece surprises her aunt with lovely drawings.",
    "Parents work hard to provide educational opportunities.",
    "Siblings share responsibility for caring for pets.",
    "The family enjoys evening walks together.",
    "The father supports his son in learning new skills.",
    "The mother cheers her daughter on at school events.",
    "The brother helps his sister study for exams.",
    "The sister encourages her brother’s hobbies.",
    "The son builds bird feeders for the backyard.",
    "The daughter organizes family photo sessions.",
    "The grandfather shares wisdom about life’s challenges.",
    "The grandmother bakes special treats for holidays.",
    "The uncle coaches his nephew’s sports team.",
    "The aunt teaches her nieces crafting skills.",
    "The cousins study together for important tests.",
    "The nephew writes letters to his grandparents.",
    "The niece paints pictures of their home.",
    "Parents teach their children values of kindness and courage.",
    "Siblings plan family activities and outings.",
    "The family holds annual reunions to stay connected.",
    "The father volunteers in community events with his son.",
    "The mother plans educational trips for her daughter.",
    "The brother teaches his sister how to ride a skateboard.",
    "The sister bakes cakes with her brother for friends.",
    "The son assists with fixing household appliances.",
    "The daughter waters plants in the greenhouse.",
    "The grandfather reminisces about his youth with laughter.",
    "The grandmother sews quilts filled with family memories.",
    "The uncle teaches first aid to his nephew.",
    "The aunt plans surprise parties for family members.",
    "The cousins explore museums and cultural landmarks.",
    "The nephew builds wooden toys for younger children.",
    "The niece decorates cakes for family celebrations.",
    "Parents encourage open communication within the family.",
    "Siblings respect each other’s privacy and feelings.",
    "The family volunteers together at local shelters.",
    "The father mentors his son in leadership skills.",
    "The mother fosters creativity in her daughter.",
    "The brother shares his collection of books.",
    "The sister sings duets with her brother.",
    "The son helps family members with technology.",
    "The daughter organizes family schedules efficiently.",
    "The grandfather leads family prayers during gatherings.",
    "The grandmother teaches traditional dances to children.",
    "The uncle supports his nephew during difficult times.",
    "The aunt entertains children with storytelling.",
    "The cousins participate in community theater performances.",
    "The nephew surprises his family with handmade gifts.",
    "The niece writes stories to share with siblings.",
    "Parents create family traditions that last for generations.",
    "Siblings celebrate each other’s successes wholeheartedly.",
    "The family enjoys cooking meals together on weekends.",
    "The father helps his son build a treehouse.",
    "The mother reads bedtime stories each night.",
    "The brother and sister practice musical instruments together.",
    "The son cares for the family garden.",
    "The daughter decorates the home during holidays.",
    "The grandfather shares lessons about resilience.",
    "The grandmother knits scarves for the winter.",
    "The uncle teaches fishing techniques to his nephew.",
    "The aunt organizes family game nights.",
    "The cousins help one another with school projects.",
    "The nephew paints portraits of family members.",
    "The niece bakes cookies to share with neighbors.",
    "Parents nurture their children’s talents and interests.",
    "Siblings learn life skills through shared responsibilities.",
    "The family supports each other through challenges.",
    "The father encourages his son to be honest.",
    "The mother listens to her daughter’s dreams.",
    "The brother teaches helpful habits to his sister.",
    "The sister inspires her brother’s creativity.",
    "The son participates in community service with family.",
    "The daughter keeps photo albums of family memories.",
    "The grandfather tells stories about family history.",
    "The grandmother preserves recipes passed down through generations.",
    "The uncle plans educational outings for his nephew.",
    "The aunt shares craft ideas with her nieces.",
    "The cousins build forts and play outside together.",
    "The nephew writes letters to distant relatives.",
    "The niece decorates the family home for special occasions."


]

with open('/kaggle/input/random-text-corpus/random_text_corpus.txt', 'r', encoding='utf-8') as f:
    text = f.read()

appending = sent_tokenize(text)

## Tokenization and Skip Gram Preparation

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(corpus)
#flat sequence of tokens ids
flat_seq = [word for sent in sequences for word in sent]

#generate the skipgrams paris
pairs, labels = skipgrams(flat_seq, vocabulary_size=vocab_size, window_size=2)
pairs = np.array(pairs)
labels = np.array(labels)

print(f"Number of training pairs: {len(pairs)}")

print(sequences[0])

Number of training pairs: 48988
[1, 26, 9, 2, 41]


## The Embedding and Neural Network

In [ ]:
embedding_dim = 20
#20 for more robustness, ideally should be power of 8

input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name='embedding')

target_emb = embedding(input_target)
context_emb = embedding(input_context)

dot = tf.keras.layers.Dot(axes=-1)([target_emb, context_emb])
dot = tf.keras.layers.Reshape((1,))(dot)
output = tf.keras.layers.Activation('sigmoid')(dot)

model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam')

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1753810270.431090   48741 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 20)     │     17,460 │ input_layer[0][0… │
│ (Embedding)         │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1, 1)      │          0 │ embedding[0][0],  │
│                     │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1)         │          0 │ dot[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1)         │          0 │ reshape[0][0]     │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,460 (68.20 KB)

 Trainable params: 17,460 (68.20 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.fit([pairs[:, 0], pairs[:, 1]], labels, epochs=500, batch_size=128)

Epoch 1/500


I0000 00:00:1753810271.990889   48788 service.cc:148] XLA service 0x7ae6940055a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753810271.990939   48788 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753810272.079356   48788 cuda_dnn.cc:529] Loaded cuDNN version 90300


 75/383 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6927

I0000 00:00:1753810272.337367   48788 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


383/383 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6703
Epoch 2/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5524
Epoch 3/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5388
Epoch 4/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5223
Epoch 5/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5160
Epoch 6/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5096
Epoch 7/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5035
Epoch 8/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4946
Epoch 9/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4905
Epoch 10/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4826
Epoch 11/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4785
Epoch 12/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4685
Epoch 13/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4628
Epoch 14/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4566
Epoch 15/500
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step -

In [6]:
embeddings = model.get_layer('embedding').get_weights()[0]

## Evaluating Word Embeddings

In [7]:
word_embeddings = {word: embeddings[idx] for word, idx in tokenizer.word_index.items()}

In [21]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def most_similar(word, word_embeddings, top_n=5):
    if word not in word_embeddings:
        print(f"'{word}' not found in vocabulary.")
        return []
    sims = {}
    target_vec = word_embeddings[word]
    for candidate, vec in word_embeddings.items():
        if candidate == word:
            continue
        sims[candidate] = cosine_similarity(target_vec, vec)
    sorted_words = sorted(sims.items(), key=lambda x: x[1], reverse=True)
    return sorted_words[:top_n]

print(most_similar("son", word_embeddings, 15))

[('husband', 0.6676301), ('sister', 0.5969477), ('supports', 0.5855362), ('effort', 0.58329314), ('husband’s', 0.5743269), ('his', 0.5513831), ('cousin', 0.54183483), ('helps', 0.527473), ('math', 0.4473354), ('clean', 0.4344693), ('fort', 0.43354422), ('wife', 0.42690137), ('drives', 0.4268138), ('marries', 0.4008196), ('butterfly', 0.39434674)]
